# 📊 PA Break Strategy — Chart Viewer & Backtest

Notebook này giúp:
1. **Xem chart** candlestick tương tác (zoom, scroll)
2. **Hiển thị Swing Points** (HH, LL)
3. **Hiển thị tín hiệu** Entry / SL / TP
4. **Backtest** và xem kết quả

Bạn có thể nói với Copilot về đường giá mà Copilot sẽ thấy được data.

In [ ]:
# Cell 1: Import và setup
import sys, importlib
sys.path.insert(0, '.')

# Force deep reload - remove cached modules
for mod_name in list(sys.modules.keys()):
    if mod_name in ('fetch_data', 'strategy_pa_break'):
        del sys.modules[mod_name]

import fetch_data, strategy_pa_break

from fetch_data import fetch_ohlcv, show_available_symbols
from strategy_pa_break import run_pa_break, signals_to_dataframe, print_backtest_summary
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

print(f"✅ Modules loaded! ({strategy_pa_break.__version__})")
show_available_symbols()

✅ Modules loaded! (v0.5.1 — Retest + Impulse Filter)

📋 Symbols có sẵn:

  Forex:
    • EURUSD
    • GBPUSD
    • USDJPY
    • AUDUSD
    • USDCAD
    • USDCHF

  Commodities:
    • XAUUSD (Gold)
    • XAGUSD (Silver)
    • OIL

  Crypto:
    • BTCUSD
    • ETHUSD

  Indices:
    • SPX500
    • NAS100
    • DJI

📋 Timeframes:
  M1, M5, M15, M30, H1, H4, D1, W1, MN
  ⚠️ M1-M30: max 7 ngày | H1-H4: max 59 ngày | D1+: không giới hạn


In [14]:
# Cell 2: Config
SYMBOL     = "XAUUSD"
TIMEFRAME  = "M5"
BARS       = 500
PIVOT_LEN  = 5
RR_RATIO   = 2.0
IMPULSE_MULT = 1.5   # Nến break phải có thân >= 1.5x avg body (0=OFF)

df = fetch_ohlcv(SYMBOL, TIMEFRAME, BARS)
print(f"\nData shape: {df.shape}")
df.tail(10)

📊 Fetching XAUUSD (GC=F) | M5 | 500 bars...
✅ Đã lấy 476 nến | Từ 2026-02-05 05:00:00+00:00 → 2026-02-06 21:55:00+00:00

Data shape: (476, 5)


Price,Open,High,Low,Close,Volume
Datetime,,,,,
2026-02-06 21:00:00+00:00,4977.200195,4978.799805,4972.799805,4975.000000,348
2026-02-06 21:05:00+00:00,4975.299805,4981.500000,4975.299805,4976.899902,125
2026-02-06 21:10:00+00:00,4977.700195,4979.500000,4970.100098,4973.200195,195
2026-02-06 21:15:00+00:00,4973.799805,4974.500000,4971.100098,4973.899902,126
2026-02-06 21:20:00+00:00,4973.700195,4974.899902,4972.799805,4974.000000,55
2026-02-06 21:35:00+00:00,4982.799805,4988.000000,4982.799805,4985.000000,530
2026-02-06 21:40:00+00:00,4985.899902,4986.299805,4982.399902,4984.000000,273
2026-02-06 21:45:00+00:00,4983.600098,4984.399902,4981.100098,4984.399902,173
2026-02-06 21:50:00+00:00,4984.100098,4988.000000,4982.299805,4988.000000,195


In [15]:
# Cell 3: Chạy Strategy v0.5.1 (Retest + Impulse)
signals, swings = run_pa_break(df, pivot_len=PIVOT_LEN, rr_ratio=RR_RATIO, impulse_mult=IMPULSE_MULT)

print(f"🔍 Swing Points: {len(swings)}")
print(f"📈 Signals: {len(signals)}")
print_backtest_summary(signals, RR_RATIO)

🔍 Swing Points: 60
📈 Signals: 10

📊 BACKTEST SUMMARY — PA Break (RR 1:2.0)
  Total signals:    10
  Closed trades:    9
  TP hit:           0 (0.0%)
  SL hit:           1 (11.1%)
  Closed (reverse): 8
  Win / Loss:       6 / 3
  Win Rate:         66.7%
  Total PnL:        1.95 R
  Avg PnL/trade:    0.22 R
  Expectancy:       1.00 R per trade
  BUY: 6 trades | WR 83.3% | PnL 1.99 R
  SELL: 3 trades | WR 33.3% | PnL -0.04 R


In [16]:
# Cell 4: Vẽ Interactive Chart với Plotly
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    row_heights=[0.8, 0.2],
                    vertical_spacing=0.02)

# Candlestick
fig.add_trace(go.Candlestick(
    x=df.index, open=df['Open'], high=df['High'],
    low=df['Low'], close=df['Close'],
    name='Price',
    increasing_line_color='#26a69a',
    decreasing_line_color='#ef5350',
), row=1, col=1)

# Swing Highs
sh_points = [s for s in swings if s.type == 'HIGH']
if sh_points:
    fig.add_trace(go.Scatter(
        x=[s.time for s in sh_points],
        y=[s.price for s in sh_points],
        mode='markers', name='Swing High',
        marker=dict(symbol='triangle-down', size=10, color='orange'),
    ), row=1, col=1)

# Swing Lows
sl_points = [s for s in swings if s.type == 'LOW']
if sl_points:
    fig.add_trace(go.Scatter(
        x=[s.time for s in sl_points],
        y=[s.price for s in sl_points],
        mode='markers', name='Swing Low',
        marker=dict(symbol='triangle-up', size=10, color='cornflowerblue'),
    ), row=1, col=1)

# Buy Signals
buy_sigs = [s for s in signals if s.direction == 'BUY']
if buy_sigs:
    for sig in buy_sigs:
        fig.add_shape(type='line',
            x0=sig.break_time, x1=sig.time,
            y0=sig.entry, y1=sig.entry,
            line=dict(color='dodgerblue', width=1, dash='dash'), row=1, col=1)
        fig.add_shape(type='line',
            x0=sig.break_time, x1=sig.time,
            y0=sig.sl, y1=sig.sl,
            line=dict(color='yellow', width=1, dash='dash'), row=1, col=1)
        if sig.tp > 0:
            fig.add_shape(type='line',
                x0=sig.break_time, x1=sig.time,
                y0=sig.tp, y1=sig.tp,
                line=dict(color='limegreen', width=1, dash='dash'), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=[s.time for s in buy_sigs],
        y=[s.entry for s in buy_sigs],
        mode='markers+text', name='BUY',
        marker=dict(symbol='triangle-up', size=14, color='dodgerblue'),
        text=['▲BUY' for _ in buy_sigs],
        textposition='top center', textfont=dict(size=9),
    ), row=1, col=1)

# Sell Signals
sell_sigs = [s for s in signals if s.direction == 'SELL']
if sell_sigs:
    for sig in sell_sigs:
        fig.add_shape(type='line',
            x0=sig.break_time, x1=sig.time,
            y0=sig.entry, y1=sig.entry,
            line=dict(color='hotpink', width=1, dash='dash'), row=1, col=1)
        fig.add_shape(type='line',
            x0=sig.break_time, x1=sig.time,
            y0=sig.sl, y1=sig.sl,
            line=dict(color='yellow', width=1, dash='dash'), row=1, col=1)
        if sig.tp > 0:
            fig.add_shape(type='line',
                x0=sig.break_time, x1=sig.time,
                y0=sig.tp, y1=sig.tp,
                line=dict(color='limegreen', width=1, dash='dash'), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=[s.time for s in sell_sigs],
        y=[s.entry for s in sell_sigs],
        mode='markers+text', name='SELL',
        marker=dict(symbol='triangle-down', size=14, color='hotpink'),
        text=['▼SELL' for _ in sell_sigs],
        textposition='bottom center', textfont=dict(size=9),
    ), row=1, col=1)

# Volume
colors = ['#26a69a' if c >= o else '#ef5350' for o, c in zip(df['Open'], df['Close'])]
fig.add_trace(go.Bar(
    x=df.index, y=df['Volume'], name='Volume',
    marker_color=colors, opacity=0.5,
), row=2, col=1)

fig.update_layout(
    title=f'{SYMBOL} {TIMEFRAME} — PA Break Strategy (RR 1:{RR_RATIO})',
    template='plotly_dark',
    height=800,
    xaxis_rangeslider_visible=False,
    showlegend=True,
    legend=dict(x=0, y=1.12, orientation='h'),
)
fig.update_yaxes(title_text='Price', row=1, col=1)
fig.update_yaxes(title_text='Volume', row=2, col=1)
fig.show()

In [17]:
# Cell 4b: Export chart ra HTML để xem trên browser
import os
html_path = os.path.join(os.getcwd(), "chart_pa_break.html")
fig.write_html(html_path)
print(f"✅ Chart đã lưu tại: {html_path}")
print(f"   Mở file này trong browser để zoom, hover, xem chi tiết.")

✅ Chart đã lưu tại: /Users/tuan/GitProject/tradingview/MST Medio/chart_pa_break.html
   Mở file này trong browser để zoom, hover, xem chi tiết.


In [5]:
# Cell 5: Xem chi tiết các signals
sig_df = signals_to_dataframe(signals)
if not sig_df.empty:
    print(f"📋 Tất cả {len(sig_df)} signals:\n")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 200)
    display(sig_df)
else:
    print("Không có signal nào.")

📋 Tất cả 2 signals:



,Time,Direction,Entry,SL,TP,Risk,Break Point,Break Time,Confirm Time,Result,PnL (R)
0,2026-02-05 10:20:00+00:00,SELL,4876.6001,4925.7532,4778.29389,49.1531,4871.1001,2026-02-05 09:55:00+00:00,2026-02-05 09:55:00+00:00,SL,-1.0
1,2026-02-06 00:40:00+00:00,SELL,4814.6001,4825.5291,4792.74209,10.9290,4670.0000,2026-02-06 00:15:00+00:00,2026-02-06 00:15:00+00:00,TP,2.0


In [ ]:
# Cell 6: So sánh nhiều RR ratios
print("📊 So sánh RR Ratios:\n")
rr_values = [1.0, 1.5, 2.0, 2.5, 3.0]
results = []

for rr in rr_values:
    sigs, _ = run_pa_break(df, pivot_len=PIVOT_LEN, rr_ratio=rr)
    sig_df_rr = signals_to_dataframe(sigs)
    closed = sig_df_rr[sig_df_rr['Result'].isin(['TP', 'SL', 'CLOSE_REVERSE'])]
    if len(closed) > 0:
        wins = len(closed[closed['PnL (R)'] > 0])
        wr = wins / len(closed) * 100
        total_r = closed['PnL (R)'].sum()
        expectancy = wr/100 * rr - (1 - wr/100)
        results.append({
            'RR': f'1:{rr}',
            'Trades': len(closed),
            'Win Rate': f'{wr:.1f}%',
            'Total PnL (R)': f'{total_r:.2f}',
            'Avg PnL (R)': f'{closed["PnL (R)"].mean():.2f}',
            'Expectancy': f'{expectancy:.2f} R',
        })

if results:
    comparison = pd.DataFrame(results)
    display(comparison)
else:
    print("Không đủ data để so sánh.")

In [ ]:
# Cell 7: Equity Curve
if signals:
    sig_df_eq = signals_to_dataframe(signals)
    closed = sig_df_eq[sig_df_eq['Result'].isin(['TP', 'SL', 'CLOSE_REVERSE'])].copy()
    
    if len(closed) > 0:
        closed['Cumulative R'] = closed['PnL (R)'].cumsum()
        
        fig_eq = go.Figure()
        fig_eq.add_trace(go.Scatter(
            x=list(range(1, len(closed)+1)),
            y=closed['Cumulative R'].values,
            mode='lines+markers',
            name='Equity (R)',
            line=dict(color='dodgerblue', width=2),
            marker=dict(
                size=8,
                color=['green' if r > 0 else 'red' for r in closed['PnL (R)']],
            ),
        ))
        fig_eq.add_hline(y=0, line_dash='dash', line_color='gray')
        fig_eq.update_layout(
            title=f'Equity Curve — {SYMBOL} {TIMEFRAME} (RR 1:{RR_RATIO})',
            template='plotly_dark',
            xaxis_title='Trade #',
            yaxis_title='Cumulative R',
            height=400,
        )
        fig_eq.show()
    else:
        print("Chưa có lệnh đóng.")
else:
    print("Không có signal.")

---
## 🔍 Công cụ khám phá giá — Dùng để mô tả cho Copilot

Các cell bên dưới giúp bạn:
1. **Xem giá quanh 1 ngày/giờ cụ thể** → "Lúc này giá tạo HH"
2. **Tìm swing points gần 1 thời điểm** → "Swing high tại đây là break point"
3. **Zoom chart vào 1 khoảng thời gian** → xem rõ cấu trúc giá

In [ ]:
# Cell 8: 🔍 Xem giá quanh 1 thời điểm cụ thể
# ═══ THAY ĐỔI TẠI ĐÂY ═══
LOOKUP_DATE = "2026-02-06 15:00"   # Ngày/giờ bạn muốn xem (M5 cần giờ)
CONTEXT_BARS = 15                   # Số nến trước và sau

target = pd.Timestamp(LOOKUP_DATE)
idx = df.index.get_indexer([target], method='nearest')[0]
start = max(0, idx - CONTEXT_BARS)
end = min(len(df), idx + CONTEXT_BARS + 1)

context_df = df.iloc[start:end].copy()
context_df['Bar#'] = range(start, end)

print(f"📍 Xem giá quanh {LOOKUP_DATE}")
print(f"   Bar gần nhất: {df.index[idx]} (index={idx})")
print(f"   Hiển thị {CONTEXT_BARS} nến trước/sau:\n")

pd.set_option('display.float_format', lambda x: f'{x:.2f}')
display(context_df)

# Swing points gần khu vực
nearby_swings = [s for s in swings 
                 if df.index[start] <= s.time <= df.index[end-1]]
if nearby_swings:
    print(f"\n🔄 Swing points trong khu vực:")
    for s in nearby_swings:
        print(f"   {'🔺' if s.type=='HIGH' else '🔻'} {s.type}: "
              f"{s.price:.2f} @ {s.time}")
else:
    print("\n   Không có swing point trong khu vực này.")

In [6]:
# Cell 9: 📋 Danh sách TẤT CẢ Swing Points
print(f"📋 Tất cả {len(swings)} Swing Points:\n")
print(f"{'#':>3} | {'Type':>5} | {'Price':>12} | {'Time':>20} | {'Bar#':>5}")
print("-" * 65)

for i, s in enumerate(swings):
    marker = "🔺" if s.type == "HIGH" else "🔻"
    print(f"{i:3d} | {marker} {s.type:>5} | {s.price:12.2f} | "
          f"{str(s.time):>20} | {s.bar_index:5d}")

print(f"\n💡 Tip: Copy dòng cụ thể rồi nói với Copilot:")
print(f'   "Swing High #12 tại giá 2850 lúc 14:30 là break point."')
print(f'   "Entry tại Swing High #10 (2830), SL tại Swing Low #11 (2810)"')

📋 Tất cả 60 Swing Points:

  # |  Type |        Price |                 Time |  Bar#
-----------------------------------------------------------------
  0 | 🔻   LOW |      4842.00 | 2026-02-05 05:30:00+00:00 |     6
  1 | 🔺  HIGH |      4951.90 | 2026-02-05 06:45:00+00:00 |    21
  2 | 🔻   LOW |      4915.00 | 2026-02-05 06:55:00+00:00 |    23
  3 | 🔺  HIGH |      4960.20 | 2026-02-05 07:25:00+00:00 |    29
  4 | 🔻   LOW |      4914.70 | 2026-02-05 07:40:00+00:00 |    32
  5 | 🔺  HIGH |      4962.00 | 2026-02-05 08:15:00+00:00 |    39
  6 | 🔻   LOW |      4876.60 | 2026-02-05 09:20:00+00:00 |    52
  7 | 🔺  HIGH |      4916.00 | 2026-02-05 09:45:00+00:00 |    57
  8 | 🔻   LOW |      4871.10 | 2026-02-05 09:55:00+00:00 |    59
  9 | 🔻   LOW |      4868.00 | 2026-02-05 10:25:00+00:00 |    65
 10 | 🔺  HIGH |      4915.50 | 2026-02-05 11:20:00+00:00 |    76
 11 | 🔻   LOW |      4831.00 | 2026-02-05 12:25:00+00:00 |    89
 12 | 🔺  HIGH |      4893.90 | 2026-02-05 12:35:00+00:00 |    91
 13 

In [ ]:
# DEBUG: Trace tại sao không có BUY tại 12:05 ngày 06/02

# 1. Swing points khu vực 11:00-14:00 ngày 06
print("=" * 60)
print("SWING POINTS 11:00-14:00 ngày 06/02")
print("=" * 60)
for i, s in enumerate(swings):
    t = s.time
    if t.day == 6 and 10 <= t.hour <= 14:
        m = "HIGH" if s.type == "HIGH" else " LOW"
        print(f"  #{i:2d} | {m} | {s.price:10.2f} | {t.strftime('%H:%M')} | bar#{s.bar_index}")

# 2. Sequence Swing Highs 
print("\n" + "=" * 60)
print("SWING HIGHS sequence 04:00-14:00 ngày 06/02")
print("=" * 60)
shs = [s for s in swings if s.type == "HIGH" and s.time.day == 6 and 4 <= s.time.hour <= 14]
for i, s in enumerate(shs):
    prev = shs[i-1] if i > 0 else None
    hh = " <-- HH!" if (prev and s.price > prev.price) else ""
    print(f"  {s.time.strftime('%H:%M')} | {s.price:10.2f}{hh}")

# 3. OHLC 12:00-14:00
print("\n" + "=" * 60)
print("OHLC 12:00-13:30 ngay 06/02")
print("=" * 60)
for idx, row in df.iterrows():
    if idx.day == 6 and idx.hour >= 12 and idx.hour <= 13:
        print(f"  {idx.strftime('%H:%M')} | O:{row['Open']:8.1f} H:{row['High']:8.1f} L:{row['Low']:8.1f} C:{row['Close']:8.1f}")

In [ ]:
# Cell 10: 🔎 Zoom Chart vào khoảng thời gian cụ thể
# ═══ THAY ĐỔI TẠI ĐÂY ═══
ZOOM_START = "2026-02-05"  # Có thể thêm giờ: "2026-02-05 08:00"
ZOOM_END   = "2026-02-07"

zoom_df = df.loc[ZOOM_START:ZOOM_END]
zoom_swings = [s for s in swings if pd.Timestamp(ZOOM_START) <= s.time <= pd.Timestamp(ZOOM_END)]
zoom_signals = [s for s in signals if pd.Timestamp(ZOOM_START) <= s.time <= pd.Timestamp(ZOOM_END)]

fig_zoom = go.Figure()

# Candlestick
fig_zoom.add_trace(go.Candlestick(
    x=zoom_df.index, open=zoom_df['Open'], high=zoom_df['High'],
    low=zoom_df['Low'], close=zoom_df['Close'],
    name='Price',
    increasing_line_color='#26a69a',
    decreasing_line_color='#ef5350',
))

# Swing points with price labels
for s in zoom_swings:
    color = 'orange' if s.type == 'HIGH' else 'cornflowerblue'
    symbol = 'triangle-down' if s.type == 'HIGH' else 'triangle-up'
    fig_zoom.add_trace(go.Scatter(
        x=[s.time], y=[s.price],
        mode='markers+text', showlegend=False,
        marker=dict(symbol=symbol, size=12, color=color),
        text=[f"{s.price:.2f}"],
        textposition='top center' if s.type == 'HIGH' else 'bottom center',
        textfont=dict(size=9, color=color),
    ))

# Signal lines
for sig in zoom_signals:
    entry_color = 'dodgerblue' if sig.direction == 'BUY' else 'hotpink'
    fig_zoom.add_shape(type='line',
        x0=sig.break_time, x1=sig.time,
        y0=sig.entry, y1=sig.entry,
        line=dict(color=entry_color, width=2, dash='dash'))
    fig_zoom.add_shape(type='line',
        x0=sig.break_time, x1=sig.time,
        y0=sig.sl, y1=sig.sl,
        line=dict(color='yellow', width=1, dash='dash'))
    if sig.tp > 0:
        fig_zoom.add_shape(type='line',
            x0=sig.break_time, x1=sig.time,
            y0=sig.tp, y1=sig.tp,
            line=dict(color='limegreen', width=1, dash='dash'))
    fig_zoom.add_annotation(
        x=sig.time, y=sig.entry,
        text=f"{'▲BUY' if sig.direction=='BUY' else '▼SELL'} | {sig.result}",
        showarrow=True, arrowhead=2,
        font=dict(size=10, color=entry_color),
    )

fig_zoom.update_layout(
    title=f'{SYMBOL} {TIMEFRAME} | Zoom: {ZOOM_START} → {ZOOM_END}',
    template='plotly_dark',
    height=600,
    xaxis_rangeslider_visible=False,
)
fig_zoom.show()

print(f"\n📊 Khu vực {ZOOM_START} → {ZOOM_END}:")
print(f"   Nến: {len(zoom_df)} | Swings: {len(zoom_swings)} | Signals: {len(zoom_signals)}")

## 🗣️ Cách nói với Copilot

Sau khi chạy notebook, bạn đã có data. Giờ mở Copilot Chat và nói:

**Ví dụ 1:** "Nhìn vào swing points trong notebook, Swing High #5 tại 2850 lúc 14:30 tạo HH. Entry tại swing cũ, SL tại swing low trước đó. Viết logic này thành Pine Script."

**Ví dụ 2:** "Backtest cho thấy win rate chỉ 40%. Thử thêm filter EMA 50 xem cải thiện không?"

**Ví dụ 3:** "Zoom vào khoảng 2026-02-05, tôi thấy signal BUY lỗ. Phân tích tại sao?"